In [1]:
import dask.dataframe as dd
from tqdm import tqdm
import pandas as pd
import numpy as np

In [ ]:
user_ids = data.user_id.unique()
course_weeks = data.course_week.unique()

for i, user_id in tqdm(user_ids.iteritems()):
    print(user_id)
    user_rows = data.loc[(data['user_id'] == user_id), :]

    active_course_weeks = user_rows.loc[:, 'course_week'].unique()

    zero_course_weeks = course_weeks[~course_weeks.isin(active_course_weeks.compute())]

    started_week = user_rows.loc[:, 'user_started_week'].min().compute()
    last_active_week = user_rows.loc[:, 'user_last_active_week'].min().compute()

    completed_week = user_rows.loc[:, 'user_completed_week'].min().compute()

#     zero_course_weeks = zero_course_weeks.between(left=0, right=int(last_active_week))
    
    zero_course_weeks = zero_course_weeks.loc[zero_course_weeks < int(last_active_week)]
    

    zero_course_weeks = zero_course_weeks.loc[zero_course_weeks > 0]

    print(zero_course_weeks.compute())
    
    

    zero_rows_df = pd.DataFrame(
        np.zeros((zero_course_weeks.count().compute(), len(data.columns))),
        columns=data.columns
    )

    zero_rows_df.loc[:, 'user_id'] = user_id
    zero_rows_df.loc[:, 'course_week'] = zero_course_weeks
    zero_rows_df.loc[:, 'user_started_week'] = started_week
    zero_rows_df.loc[:, 'user_last_active_week'] = last_active_week
    zero_rows_df.loc[:, 'user_completed_week'] = completed_week

    
    data = dd.concat([data, zero_rows_df], axis=0, interleave_partitions=True)
#     raise ValueError('check')
    print(data)
#     data = data.append(zero_rows_df)

In [126]:
len(data.compute())

182112

In [1]:
import pandas as pd

In [16]:
data = pd.read_csv('data/Microsoft+DAT206x+3T2017/model_data_l.csv')
data['user_dropped_out_next_week'].value_counts()

0    46571
1    19500
Name: user_dropped_out_next_week, dtype: int64

In [17]:
d = pd.read_csv('data/Microsoft+DAT247x+4T2017/model_data.csv')
d_l = pd.read_csv('data/Microsoft+DAT247x+4T2017/model_data_l.csv')

In [21]:
d['user_dropped_out_next_week'].value_counts()

0    2533
1     544
Name: user_dropped_out_next_week, dtype: int64

In [22]:
d_l['user_dropped_out_next_week'].value_counts()

0    617
1    544
Name: user_dropped_out_next_week, dtype: int64

In [6]:
from sklearn.metrics import accuracy_score

In [7]:
accuracy_score(d['user_dropped_out_next_week'], d['predicted_user_dropped_out_next_week'])

0.89470263243418913

In [8]:
y_true = d['user_dropped_out_next_week']

In [9]:
y_true.shape

(3077,)

In [10]:
import numpy as np
np.zeros(y_true.shape)

array([ 0.,  0.,  0., ...,  0.,  0.,  0.])

In [11]:
accuracy_score(d['user_dropped_out_next_week'], np.zeros(y_true.shape))

0.82320441988950277

In [34]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score, confusion_matrix

In [39]:
model_accs = []
base_accs = []
precs = []
recs = []
f1s = []
t1s = []
cks = []
course_ids = []


with open('{}/top_course_ids.txt'.format('data')) as top_course_ids:
    for top_course_id in top_course_ids:
        if '4T2017' in top_course_id and 'JPN' not in top_course_id and 'DAT102x' not in top_course_id:
            print('Running for course: ', top_course_id)
            try:
                final = pd.read_csv('data/{}/model_data_with_preds_l.csv'.format(top_course_id.strip()))
                course_ids.append(top_course_id.strip())
                y_true = final['user_dropped_out_next_week']
                y_pred = final['predicted_user_dropped_out_next_week']
            except:
                print('Course: {} did not have predictions.'.format(top_course_id))
                continue
                
                
            base_acc = np.zeros(y_true.shape)


            model_accs.append(accuracy_score(y_true, y_pred))
            base_accs.append(accuracy_score(y_true, base_acc))


            precs.append(precision_score(y_true, y_pred))
            recs.append(recall_score(y_true, y_pred))
            
            f1s.append(f1_score(y_true, y_pred))

            conf_matrix = confusion_matrix(y_true, y_pred)
            t1s.append(conf_matrix[1][0] / len(y_true))

            cks.append(cohen_kappa_score(y_true, y_pred))
                
            
            

Running for course:  Microsoft+DAT101x+4T2017

Running for course:  Microsoft+DAT201x+4T2017

Course: Microsoft+DAT201x+4T2017
 did not have predictions.
Running for course:  Microsoft+DAT202.1x+4T2017

Running for course:  Microsoft+DAT202.2x+4T2017

Running for course:  Microsoft+DAT202.3x+4T2017

Running for course:  Microsoft+DAT203.1x+4T2017

Running for course:  Microsoft+DAT203.2x+4T2017

Running for course:  Microsoft+DAT203.3x+4T2017

Running for course:  Microsoft+DAT204x+4T2017

Running for course:  Microsoft+DAT205x+4T2017

Running for course:  Microsoft+DAT206x+4T2017

Running for course:  Microsoft+DAT207x+4T2017

Running for course:  Microsoft+DAT208x+4T2017

Running for course:  Microsoft+DAT209x+4T2017

Running for course:  Microsoft+DAT210x+4T2017

Running for course:  Microsoft+DAT211x+4T2017

Running for course:  Microsoft+DAT212x+4T2017

Running for course:  Microsoft+DAT213x+4T2017

Running for course:  Microsoft+DAT215.2x+4T2017

Running for course:  Microsoft+DA

In [40]:
scores = pd.DataFrame(
    [],
    columns=[
        'course_id', 'model_accuracy', 'base_accuracy (all 0)',
        'precision', 'recall', 'f1_score', 'num_false_negatives',
        'cohens_kappa'
    ]
)

In [41]:
scores['course_id'] = course_ids
scores['model_accuracy'] = model_accs
scores['base_accuracy (all 0)'] = base_accs
# scores['precision'] = precs
scores['recall'] = recs
scores['f1_score'] = f1s
scores['false_neg_perc'] = t1s
scores['cohens_kappa'] = cks

In [47]:
scores.describe()

,model_accuracy,base_accuracy (all 0),recall,f1_score,cohens_kappa,false_neg_perc
count,34.000000,34.000000,34.000000,34.000000,34.000000,34.000000
mean,0.674961,0.648281,0.983410,0.671612,0.402177,0.005380
std,0.047780,0.084610,0.015417,0.079014,0.065769,0.004529
min,0.587410,0.448174,0.922293,0.535013,0.288431,0.000000
25%,0.635616,0.600205,0.980293,0.616322,0.350971,0.003111
50%,0.667511,0.672507,0.987569,0.657782,0.396413,0.004149
75%,0.709022,0.701230,0.992316,0.731576,0.447298,0.006109
max,0.761621,0.774647,1.000000,0.809558,0.531047,0.024187


In [104]:
from cntk.device import try_set_default_device, gpu
try_set_default_device(gpu(0))

True

In [ ]:
from collections import Counter
import numpy as np
import pandas as pd

from cntk.device import try_set_default_device, gpu
try_set_default_device(gpu(0))

import keras
from keras.models import Sequential, load_model
from keras.layers import Activation, Dense, Dropout
from keras.layers.normalization import BatchNormalization
from keras import optimizers
from keras.utils import plot_model
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
# from sklearn.cross_validation import train_test_split
from pipeline.util import *


def get_data_path():
    return './data'

def save_df_to_file(dataframe, name, course_id, type='csv'):
    """
    Save a dataframe to a csv file in the data/{course_id}/ directory
    """

    path = '{}/{}/{}'.format(get_data_path(), course_id, name)

    if type == 'excel':
        path += '.xlsx'
        create_directory_safe(path)
        dataframe.to_excel(path)
    else:
        path += '.csv'
        create_directory_safe(path)
        dataframe.to_csv(path, index=False)
        
        
def get_data(current_course_id):
    """
    TODO Fix how this training data is sampled
    e.g. bootstrap sampling of a random number of courses
    to get a total of > 1 million training samples
    """

    # train = pd.read_csv('{}/{}/model_data.csv'.format(get_data_path(), 'Microsoft+DAT206x+3T2017'))
    train = None
    past_course_ids = [f for f in os.listdir(get_data_path()) if not f.startswith('.')]
    try:
        past_course_ids.remove(current_course_id)
    except ValueError:
        print('Not in list')

    for course_id in past_course_ids:
        if '4T2017' not in course_id:
            try:
                # course_run_data = pd.read_csv('{}/{}/model_data.csv'.format(get_data_path(), course_id))
                path = '{}/{}/model_data_l.csv'.format(get_data_path(), course_id)
                course_run_data = pd.read_csv(path)
            except Exception:
                print('model_data.csv does not exist for course: ', course_id)
                continue
                # pass                
            if train is None:
                train = course_run_data
            else:
                train = train.append(course_run_data)

    print('Training data done.')

    train = train.reset_index(drop=True)
    # test = pd.read_csv('{}/{}/model_data.csv'.format(get_data_path(), current_course_id))
    test = pd.read_csv('{}/{}/model_data_l.csv'.format(get_data_path(), current_course_id))

#     print('SHAPE CHECK')
#     print(train.shape)
#     train = fix_output_class_values(train)
#     print(train.shape)
#     test = fix_output_class_values(test)
    
    
    X_cols = [
        'course_week', 'num_video_plays', 'num_problems_attempted',
        'num_problems_correct', 'num_subsections_viewed', 'num_forum_posts',
        'num_forum_votes', 'avg_forum_sentiment', 
        'user_started_week', 'user_active_previous_week'
    ]

    scaler = StandardScaler()
    scaler.fit(train[X_cols])

    X_train = scaler.transform(train[X_cols])
    X_test = scaler.transform(test[X_cols])

    X_train = np.array(X_train).astype(np.float32)
    X_test = np.array(X_test).astype(np.float32)

    y_train = np.array(train['user_dropped_out_next_week']).astype(np.float32)
    y_test = np.array(test['user_dropped_out_next_week']).astype(np.float32)

    return (X_train, y_train, X_test, y_test)


def _create_pivot_table(df, val_col):
    df_pivot = df.pivot_table(
        index='user_id', columns=['course_week'], values=val_col, fill_value=-1
    )
    df_colored = df_pivot.style.applymap(_cell_colors)
    return df_colored

def _cell_colors(s):
    ret = 'background-color: {}'
    if s == 0:
        ret = ret.format('#228b22')
    elif s == 1:
        ret = ret.format('#dc143c')
    else:
        ret = ret.format('#d3d3d3')

    return ret

# Function to create model, required for KerasClassifier
def create_model(optimizer='adam', hidden_layers_dim=8):
    # create model
    model = Sequential()
    model.add(Dense(hidden_layers_dim, input_dim=10))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    
    model.add(Dense(8))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    
    model.add(Dense(6))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    
    model.add(Dense(2))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    
    model.add(Dense(1))
    model.add(BatchNormalization())
    model.add(Activation('sigmoid'))

    
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model


def fit_score_predict(course_id, train=False):

    print('GETTING DATA: ')
    X_train, y_train, X_test, y_test = get_data(course_id)
    print('Done.')

    batch_size = 256
    
    if not train:
        current_date_string = datetime.strftime(datetime.today(), '%Y-%m-%d')
        model = load_model('model.h5-{}'.format(current_date_string))
    else:
        adam = optimizers.Adam(lr=0.01)

        print('Fitting model')
        
        kfold = StratifiedKFold(n_splits=10, shuffle=True)
        cv_scores = []
        for train_ind, val_ind in kfold.split(X_train, y_train):
            model = create_model(optimizer=adam)
            model.fit(X_train[train_ind], 
                      y_train[train_ind], 
                      epochs=3, 
                      batch_size=batch_size, 
                      class_weight={ 0: 1., 1: 2 },                      
                      verbose=1)
            
            scores = model.evaluate(X_train[val_ind], y_train[val_ind], batch_size)
            print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
            cv_scores.append(scores[1] * 100)
        print("%.2f%% (+/- %.2f%%)" % (numpy.mean(cv_scores), numpy.std(cv_scores)))
        print('Done')
        
        try:
            current_date_string = datetime.strftime(datetime.today(), '%Y-%m-%d')
            model.save('model.h5-{}'.format(current_date_string))
        except:
            print('FAILED TO SAVE MODEL')

    print('Evaluating model on data for course: {}'.format(course_id))

    score = model.evaluate(X_test, y_test, batch_size)
    print('Model score', score)

    preds = model.predict(X_test, batch_size)
    final_preds = np.round(preds)

    print('PREDS: ', final_preds)
    print('Y_TEST: ', y_test)

    conf_matrix = metrics.confusion_matrix(y_test, final_preds)

    tn, fp, fn, tp = conf_matrix.ravel()
    total = len(y_test)
    final_acc = (tn + tp) / total

    test_data_orig = pd.read_csv('{}/{}/model_data_l.csv'.format(get_data_path(), course_id))
    test_data_orig = fix_output_class_values(test_data_orig)
    test_data_orig['predicted_user_dropped_out_next_week'] = final_preds
    

    pred_pivot = _create_pivot_table(test_data_orig, 'predicted_user_dropped_out_next_week')
    real_pivot = _create_pivot_table(test_data_orig, 'user_dropped_out_next_week')

    # save_df_to_file(pred_pivot, 'predicted_dropouts', course_id, type='excel')
    # save_df_to_file(real_pivot, 'real_dropouts', course_id, type='excel')
    # save_df_to_file(test_data_orig, 'model_data_with_preds', course_id)

#     save_df_to_file(pred_pivot, 'predicted_dropouts_l', course_id, type='excel')
#     save_df_to_file(real_pivot, 'real_dropouts_l', course_id, type='excel')
    save_df_to_file(test_data_orig, 'model_data_with_preds_l', course_id)

    print('ACCURACY: ', final_acc)
    print('CONFUSION MATRIX: ')
    print(conf_matrix)
    print(conf_matrix / len(y_test))

    return (final_preds, final_acc, conf_matrix)


In [ ]:
fit_score_predict('Microsoft+DAT247x+4T2017', train=True)

GETTING DATA: 
model_data.csv does not exist for course:  Microsoft+DAT215.4x+1T2017
model_data.csv does not exist for course:  _SUCCESS
model_data.csv does not exist for course:  Microsoft+DAT206x+JPN+1T2017
model_data.csv does not exist for course:  Microsoft+DAT206x+6T2016
model_data.csv does not exist for course:  Microsoft+DAT206x+1T2018
model_data.csv does not exist for course:  Microsoft+DAT215.3x+3T2017
model_data.csv does not exist for course:  Microsoft+DAT206x+JPN+2T2017
model_data.csv does not exist for course:  Microsoft+DAT207x+1T2018
model_data.csv does not exist for course:  top_course_ids.txt
model_data.csv does not exist for course:  Microsoft+DAT205x+3T2016
Training data done.
Done.
Fitting model


/home/kabirkhan/Documents/ML_Experiments/py35/lib/python3.5/site-packages/keras/backend/cntk_backend.py:2337: UserWarning: CNTK backend warning: CNTK version not detected. Will using CNTK 2.0 GA as default.
  'CNTK backend warning: CNTK version not detected. '


Epoch 1/3
   1792/1779013 [..............................] - ETA: 4:15 - loss: 1.1920 - acc: 0.4782 

/home/kabirkhan/Documents/ML_Experiments/py35/lib/python3.5/site-packages/cntk/core.py:361: UserWarning: your data is of type "float64", but your input variable (uid "Input54272") expects "<class 'numpy.float32'>". Please convert your data beforehand to speed up training.
  (sample.dtype, var.uid, str(var.dtype)))


1779013/1779013 [==============================] - 61s 35us/step - loss: 0.6798 - acc: 0.7754
Epoch 2/3
1779013/1779013 [==============================] - 63s 35us/step - loss: 0.6680 - acc: 0.7762
Epoch 3/3
197669/197669 [==============================] - 2s 8us/step
acc: 80.75%
Epoch 1/3
   1792/1779013 [..............................] - ETA: 4:00 - loss: 0.9965 - acc: 0.6468 

...

1779014/1779014 [==============================] - 61s 34us/step - loss: 0.6678 - acc: 0.7407
Epoch 2/3
1779014/1779014 [==============================] - 61s 34us/step - loss: 0.6528 - acc: 0.7394
Epoch 3/3
197668/197668 [==============================] - 2s 8us/step
acc: 77.27%
Epoch 1/3
   1536/1779014 [..............................] - ETA: 4:43 - loss: 1.0733 - acc: 0.5964 

/home/kabirkhan/Documents/ML_Experiments/py35/lib/python3.5/site-packages/cntk/core.py:361: UserWarning: your data is of type "float64", but your input variable (uid "Input78009") expects "<class 'numpy.float32'>". Please convert your data beforehand to speed up training.
  (sample.dtype, var.uid, str(var.dtype)))


1779014/1779014 [==============================] - 61s 35us/step - loss: 0.6771 - acc: 0.7711
Epoch 2/3
1779014/1779014 [==============================] - 61s 34us/step - loss: 0.6718 - acc: 0.7725
Epoch 3/3
197668/197668 [==============================] - 2s 8us/step
acc: 80.44%
Epoch 1/3
    256/1779015 [..............................] - ETA: 23:42 - loss: 1.1375 - acc: 0.5156

/home/kabirkhan/Documents/ML_Experiments/py35/lib/python3.5/site-packages/cntk/core.py:361: UserWarning: your data is of type "float64", but your input variable (uid "Input81400") expects "<class 'numpy.float32'>". Please convert your data beforehand to speed up training.
  (sample.dtype, var.uid, str(var.dtype)))


1758976/1779015 [============================>.] - ETA: 0s - loss: 0.6665 - acc: 0.7367

In [ ]:
data_ = pd.read_csv('./data/Microsoft+DAT206x+1T2018/model_data_with_preds_l.csv')

In [134]:
data_.tail(8)

,user_id,course_week,num_video_plays,num_problems_attempted,num_problems_correct,num_subsections_viewed,num_forum_posts,num_forum_votes,avg_forum_sentiment,user_started_week,user_last_active_week,user_completed_week,user_dropped_out_next_week,predicted_user_dropped_out_next_week
1153,17004716,9.0,100.0,18.0,6.0,32.0,0.0,0.0,0.0,9.0,12.0,-1.0,0,0.0
1154,17004716,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,12.0,-1.0,1,1.0
1155,17004716,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,12.0,-1.0,1,1.0
1156,17004716,12.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,9.0,12.0,-1.0,1,1.0
1157,17010776,9.0,6.0,1.0,1.0,11.0,0.0,0.0,0.0,9.0,11.0,-1.0,0,1.0
1158,17010776,10.0,60.0,0.0,0.0,8.0,0.0,0.0,0.0,9.0,11.0,-1.0,0,0.0
1159,17010776,11.0,48.0,0.0,0.0,13.0,0.0,0.0,0.0,9.0,11.0,-1.0,1,0.0
1160,17016359,9.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,9.0,9.0,-1.0,1,1.0


In [148]:
data_[data_['course_week'] - 1 == data_['user_started_week']]

,user_id,course_week,num_video_plays,num_problems_attempted,num_problems_correct,num_subsections_viewed,num_forum_posts,num_forum_votes,avg_forum_sentiment,user_started_week,user_last_active_week,user_completed_week,user_dropped_out_next_week,predicted_user_dropped_out_next_week
4,313947,2.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,1.0,14.0,-1.0,1,1.0
19,397135,7.0,0.0,0.0,0.0,1.0,0.000,0.0,0.0,6.0,7.0,-1.0,1,1.0
22,423439,5.0,0.0,0.0,0.0,7.0,0.000,0.0,0.0,4.0,10.0,-1.0,0,1.0
32,534519,2.0,95.0,25.0,25.0,24.0,0.000,0.0,0.0,1.0,2.0,2.0,0,0.0
42,1171039,4.0,56.0,31.0,18.0,17.0,0.000,0.0,0.0,3.0,10.0,-1.0,0,0.0
52,1339772,2.0,2.0,1.0,1.0,6.0,0.000,0.0,0.0,1.0,12.0,-1.0,0,1.0
67,1783482,8.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,7.0,13.0,-1.0,1,1.0
75,1873810,7.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,6.0,8.0,-1.0,1,1.0
78,1962815,11.0,0.0,0.0,0.0,1.0,0.000,0.0,0.0,10.0,15.0,-1.0,0,1.0
84,2437683,5.0,13.0,6.0,4.0,23.0,0.000,0.0,0.0,4.0,6.0,6.0,0,0.0


In [136]:
events = pd.read_csv('./data/Microsoft+DAT206x+1T2018/events.csv')

/home/kabirkhan/Documents/ML_Experiments/py35/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [152]:
course_id = 'Microsoft+DAT206x+1T2018'

In [150]:
features = pd.read_csv('./data/Microsoft+DAT206x+1T2018/features.csv')

In [172]:
import logging
import pandas as pd
import numpy as np
from pipeline.util import *
from tqdm import tqdm

LOG_CONFIG = {'root':{'handlers':('console', 'file'), 'level':'DEBUG'}}

FORMAT = '%(name)s:   %(message)s'
logging.basicConfig(format=FORMAT, level=logging.INFO)
LOG = logging.getLogger(__name__)


def add_neg_data_points(course_id, features, from_checkpoint=False):
    """
    Add negative data points to a features dataframe
    """
    if from_checkpoint:
        def get_data_from_file(name):
            return pd.read_csv("{}/{}/{}.csv".format(get_data_path(), course_id, name))
        # data = get_data_from_file('model_data')
        data = get_data_from_file('model_data_l')
    else:
        LOG.info('Adding negative data points for course: {}'.format(course_id))

        data = features.copy()

        user_ids = pd.unique(data['user_id'])
        course_weeks = set(pd.unique(data['course_week']))

        users_completed_count = 0
        # model_data_path = '{}/{}/model_data.csv'.format(get_data_path(), course_id)

        for user_id in tqdm(user_ids[0:20]):
            user_rows = data.loc[(data['user_id'] == user_id), :]

            active_course_weeks = set(pd.unique(user_rows.loc[:, 'course_week']))
            zero_course_weeks = list(course_weeks - active_course_weeks)

            started_week = user_rows.loc[:, 'user_started_week'].min()
            last_active_week = user_rows.loc[:, 'user_last_active_week'].min()
            completed_week = user_rows.loc[:, 'user_completed_week'].min()

            print(started_week, last_active_week)
            
            # zero_course_weeks = [x for x in zero_course_weeks if (x < last_active_week and x > 0)]
            zero_course_weeks = [x for x in zero_course_weeks if (x < last_active_week and x > started_week)]

            zero_rows_df = pd.DataFrame(
                np.zeros((len(zero_course_weeks), len(data.columns))),
                columns=data.columns
            )

            zero_rows_df.loc[:, 'user_id'] = user_id
            zero_rows_df.loc[:, 'course_week'] = zero_course_weeks
            zero_rows_df.loc[:, 'user_started_week'] = started_week
            zero_rows_df.loc[:, 'user_last_active_week'] = last_active_week
            zero_rows_df.loc[:, 'user_completed_week'] = completed_week

            print(zero_rows_df.head())
            
            data = data.append(zero_rows_df)
            data = data.reset_index(drop=True)            
            
            print('============================================')
            
            
        raise ValueError('STOPPING')
            
        data = data.sort_values(['user_id', 'course_week']).reset_index(drop=True)
        data = data.fillna(0.0)

        LOG.info('Done')
        LOG.info('Calculating user dropped out next week.')
        data = calculate_drop_out_next_week(data)
        LOG.info('Done')

        LOG.info('Saving to csv...')
        # save_df_to_file(data, 'model_data', course_id)
#         save_df_to_file(data, 'model_data_l', course_id)
        LOG.info('Done')
#         del data

    return data


def calculate_drop_out_next_week(features):
    """
    Calculate user_dropped_out_next_week value (this is the value we will try to predict)
    """
    data_ = features.copy()
    data_['user_dropped_out_next_week'] = np.where(
        data_['user_last_active_week'] == data_['course_week'], 1, 0
    )
    
    data_.loc[data_['user_completed_week'] > -1, 'user_dropped_out_next_week'] = 0
    return data_
    

In [173]:
add_neg_data_points(course_id, features)

__main__:   Adding negative data points for course: Microsoft+DAT206x+1T2018
100%|██████████| 20/20 [00:00<00:00, 167.48it/s]

1 -1 -1
Empty DataFrame
Columns: [user_id, course_week, num_video_plays, num_problems_attempted, num_problems_correct, num_subsections_viewed, num_forum_posts, num_forum_votes, avg_forum_sentiment, user_started_week, user_last_active_week, user_completed_week]
Index: []
1 1 -1
Empty DataFrame
Columns: [user_id, course_week, num_video_plays, num_problems_attempted, num_problems_correct, num_subsections_viewed, num_forum_posts, num_forum_votes, avg_forum_sentiment, user_started_week, user_last_active_week, user_completed_week]
Index: []
1 1 -1
Empty DataFrame
Columns: [user_id, course_week, num_video_plays, num_problems_attempted, num_problems_correct, num_subsections_viewed, num_forum_posts, num_forum_votes, avg_forum_sentiment, user_started_week, user_last_active_week, user_completed_week]
Index: []
1 1 -1
Empty DataFrame
Columns: [user_id, course_week, num_video_plays, num_problems_attempted, num_problems_correct, num_subsections_viewed, num_forum_posts, num_forum_votes, avg_forum_sen

ValueError: STOPPING

In [177]:
data_.head(10)

,user_id,course_week,num_video_plays,num_problems_attempted,num_problems_correct,num_subsections_viewed,num_forum_posts,num_forum_votes,avg_forum_sentiment,user_started_week,user_last_active_week,user_completed_week,user_dropped_out_next_week,predicted_user_dropped_out_next_week
0,50793,5.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,5.0,5.0,-1.0,1,1.0
1,92307,8.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,8.0,8.0,-1.0,1,1.0
2,209746,11.0,2.0,1.0,0.0,6.0,0.0,0.0,0.0,11.0,11.0,-1.0,1,1.0
3,269550,3.0,14.0,0.0,0.0,8.0,0.0,0.0,0.0,3.0,3.0,-1.0,1,1.0
4,313947,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,14.0,-1.0,1,1.0
5,313947,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,14.0,-1.0,1,1.0
6,313947,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,14.0,-1.0,1,1.0
7,313947,5.0,11.0,0.0,0.0,4.0,0.0,0.0,0.0,1.0,14.0,-1.0,0,1.0
8,313947,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,14.0,-1.0,1,1.0
9,313947,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,14.0,-1.0,1,1.0


In [189]:
w = pd.DataFrame(data_['user_id'].value_counts())
w = w[w > 1].dropna()

In [191]:
data_.set_index('user_id').loc[w.index]

,course_week,num_video_plays,num_problems_attempted,num_problems_correct,num_subsections_viewed,num_forum_posts,num_forum_votes,avg_forum_sentiment,user_started_week,user_last_active_week,user_completed_week,user_dropped_out_next_week,predicted_user_dropped_out_next_week
user_id,,,,,,,,,,,,,
6825354,1.0,1.0,0.0,0.0,4.0,0.0,0.0,0.0,1.0,15.0,-1.0,0,1.0
6825354,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,15.0,-1.0,1,1.0
6825354,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,15.0,-1.0,1,1.0
6825354,4.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,1.0,15.0,-1.0,0,1.0
6825354,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,15.0,-1.0,1,1.0
6825354,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,15.0,-1.0,1,1.0
6825354,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,15.0,-1.0,1,1.0
6825354,8.0,5.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,15.0,-1.0,0,1.0
6825354,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,15.0,-1.0,1,1.0


In [282]:
shifted = data_.set_index(['user_id', 'course_week']).groupby(level=0).shift(1)[cols].reset_index()

In [285]:
shifted = data_.set_index(['user_id', 'course_week']).groupby(level=0).shift(1)[cols].reset_index()
shifted.columns = ['user_id', 'course_week', 'prev_num_video_plays', 'prev_num_problems_attempted',
       'prev_num_problems_correct', 'prev_num_subsections_viewed', 'prev_num_forum_posts',
       'prev_num_forum_votes', 'prev_avg_forum_sentiment']

data_ = pd.merge(data_, shifted, on=['user_id', 'course_week'])

In [288]:
data_ = pd.merge(data_, shifted, on=['user_id', 'course_week'])

In [292]:
data_

,user_id,course_week,num_video_plays,num_problems_attempted,num_problems_correct,num_subsections_viewed,num_forum_posts,num_forum_votes,avg_forum_sentiment,user_started_week,...,user_completed_week,user_dropped_out_next_week,predicted_user_dropped_out_next_week,prev_num_video_plays,prev_num_problems_attempted,prev_num_problems_correct,prev_num_subsections_viewed,prev_num_forum_posts,prev_num_forum_votes,prev_avg_forum_sentiment
0,50793,5.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,5.0,...,-1.0,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,92307,8.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,8.0,...,-1.0,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,209746,11.0,2.0,1.0,0.0,6.0,0.0,0.0,0.0,11.0,...,-1.0,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,269550,3.0,14.0,0.0,0.0,8.0,0.0,0.0,0.0,3.0,...,-1.0,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,313947,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,-1.0,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,313947,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,-1.0,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,313947,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,-1.0,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,313947,5.0,11.0,0.0,0.0,4.0,0.0,0.0,0.0,1.0,...,-1.0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,313947,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,-1.0,1,1.0,11.0,0.0,0.0,4.0,0.0,0.0,0.0
9,313947,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,-1.0,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [289]:
prev_cols = ['prev_num_video_plays', 'prev_num_problems_attempted',
       'prev_num_problems_correct', 'prev_num_subsections_viewed', 'prev_num_forum_posts',
       'prev_num_forum_votes', 'prev_avg_forum_sentiment']

In [293]:
data_[data_[prev_cols] == 0]

,user_id,course_week,num_video_plays,num_problems_attempted,num_problems_correct,num_subsections_viewed,num_forum_posts,num_forum_votes,avg_forum_sentiment,user_started_week,...,user_completed_week,user_dropped_out_next_week,predicted_user_dropped_out_next_week,prev_num_video_plays,prev_num_problems_attempted,prev_num_problems_correct,prev_num_subsections_viewed,prev_num_forum_posts,prev_num_forum_votes,prev_avg_forum_sentiment


In [322]:
prev_ranked = data_[prev_cols]
prev_ranked[prev_ranked != 0].dropna(thresh=1).index

data_.loc[prev_ranked[prev_ranked != 0].dropna(thresh=1).index, 'active_previous_week'] = 1.

In [325]:
data_ = data_['active_previous_week'].fillna(0.)

In [258]:
t = None
for a, b in g_data:
    user_id, course_week = a
    print(course_week)
    prev_course_week = course_week - 1.
    t = data_.loc[(data_['user_id'] == user_id) & (data_['course_week'] == course_week), :]
    s = data_.loc[(data_['user_id'] == user_id) & (data_['course_week'] == prev_course_week), :]
    
    ranked = s[cols]
    print('============================================')
    print(s.loc[ranked[ranked == 0].dropna().index, cols])
    active_prev_week = s.loc[ranked[ranked != 0].dropna().index]
    if not active_prev_week.empty:
        t['active_previous_week'] = 1.
    else:
        t['active_previous_week'] = 0.
    
    raise ValueError('STOP')
#     t = b
# t.head()

5.0
Empty DataFrame
Columns: [num_video_plays, num_problems_attempted, num_problems_correct, num_subsections_viewed, num_forum_posts, num_forum_votes, avg_forum_sentiment]
Index: []


/home/kabirkhan/Documents/ML_Experiments/py35/lib/python3.5/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


ValueError: STOP

In [243]:
data_[(data_['user_id'] == 50793) & (data_['course_week'] == 5.0)]

,user_id,course_week,num_video_plays,num_problems_attempted,num_problems_correct,num_subsections_viewed,num_forum_posts,num_forum_votes,avg_forum_sentiment,user_started_week,user_last_active_week,user_completed_week,user_dropped_out_next_week,predicted_user_dropped_out_next_week
0,50793,5.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,5.0,5.0,-1.0,1,1.0


In [234]:
t.head()

,user_id,course_week,num_video_plays,num_problems_attempted,num_problems_correct,num_subsections_viewed,num_forum_posts,num_forum_votes,avg_forum_sentiment,user_started_week,user_last_active_week,user_completed_week,user_dropped_out_next_week,predicted_user_dropped_out_next_week
0,50793,5.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,5.0,5.0,-1.0,1,1.0


In [197]:
cols = ['num_video_plays', 'num_problems_attempted',
           'num_problems_correct', 'num_subsections_viewed', 'num_forum_posts',
           'num_forum_votes', 'avg_forum_sentiment']

ranked = data_[cols]

data_.loc[ranked[ranked == 0].dropna().index, cols].groupby('user_id')

,num_video_plays,num_problems_attempted,num_problems_correct,num_subsections_viewed,num_forum_posts,num_forum_votes,avg_forum_sentiment
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14,0.0,0.0,0.0,0.0,0.0,0.0,0.0
